# Evaluation

In [1]:
import sys
import os

repo_path = os.path.sep.join(os.getcwd().split(os.path.sep)[:-3])
print(repo_path)
sys.path.append(repo_path)
%cd ../../..

/home/loebbert/projects/deepclustering/LMU_Master_Practical_SoSe24
/home/loebbert/projects/deepclustering/LMU_Master_Practical_SoSe24


In [2]:
from clustpy.deep.autoencoders import FeedforwardAutoencoder
from practical.DeepClustering.DeepECT.evaluation_pipeline import DatasetType, evaluate

[WARNING] Could not import cv2 in clustpy.data.real_video_data. Please install cv2 by 'pip install opencv-python' if necessary


## Flat Clustering

### MNIST

In [4]:
flat, _ = evaluate(
    init_autoencoder=FeedforwardAutoencoder, dataset_type=DatasetType.MNIST, seed=42
)
print(flat)

Autoencoder parameters loaded from file.
cuda


Fit:   1%|          | 530/50000 [00:02<11:24, 72.23it/s] 

Leaf assignments: 48506
Tree size after growing: 5, leaf nodes: 3


Fit:   2%|▏         | 1037/50000 [00:05<09:34, 85.26it/s]

Leaf assignments: 30446
Tree size after growing: 7, leaf nodes: 4


...

Leaf assignments: 6544
Tree size after growing: 39, leaf nodes: 20


Predict: 100%|██████████| 274/274 [00:00<00:00, 358.58it/s]


fitting IDEC...


/home/loebbert/miniconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


finished fitting IDEC
fitting KMeans...
finished fitting Kmeans
  dataset                  method       nmi       acc       ari        dp  \
0   MNIST                 DeepECT  0.803724  0.848771  0.762642  0.804942   
1   MNIST  DeepECT + Augmentation  0.000000  0.000000  0.000000       NaN   
2   MNIST                    IDEC  0.770469  0.830171  0.719716       NaN   
3   MNIST                  KMeans  0.770853  0.830629  0.720985       NaN   

       lp  seed  
0  0.9336    42  
1     NaN    42  
2     NaN    42  
3     NaN    42  


### USPS

In [ ]:
evaluate(
    init_autoencoder=FeedforwardAutoencoder, dataset_type=DatasetType.USPS, seed=42
)

### Fashion-MNIST

In [ ]:
evaluate(
    init_autoencoder=FeedforwardAutoencoder, dataset_type=DatasetType.FASHION_MNIST, seed=42
)

### Reuters

In [ ]:
evaluate(
    init_autoencoder=FeedforwardAutoencoder, dataset_type=DatasetType.REUTERS, seed=42
)

## Hierarchical Clustering

### MNIST

### USPS

### Fashion-MNIST

### Reuters